In [5]:
import aiohttp

In [6]:
import aiohttp
async def main():
    async with aiohttp.ClientSession() as session:
        async with session.get('https://geoqa.di.uoa.gr/geospatial_gold_standard.html') as response:

            print("Status:", response.status)
            print("Content-type:", response.headers['content-type'])

            html = await response.text()
            return html

In [7]:
html = await main()

Status: 200
Content-type: text/html


In [8]:
import lxml, cssselect

In [9]:
import lxml.html
tree = lxml.html.document_fromstring(html)

In [27]:
import re

RE_URL= re.compile(r"assets/queries/\d+\.sparql$")
RE_QUESTION = re.compile(r"^(\d+)\.\s+?(.*)$")

def parse_link(elem, root_url="https://geoqa.di.uoa.gr/"):
    text = str(elem.text_content()).strip()
    match = RE_QUESTION.match(text)
    href = elem.attrib["href"]
    return {"href": f"{root_url}{href}", "number": int(match.group(1)), "question": match.group(2)}

questions = [parse_link(a) for a in tree.cssselect('a[href^="assets/queries"]') if RE_URL.search(a.attrib["href"])]

In [34]:
async def get_answers(questions): 
    final_questions = []
    async with aiohttp.ClientSession() as session:
        for q in questions:
            async with session.get(q["href"]) as response:
                sparql = await response.text()
                final_questions.append({**q, "answer": sparql})
    return final_questions


In [35]:
final_questions = await get_answers(questions)

In [38]:
import json
with open("geoqa201.json", "w") as f:
    json.dump(final_questions, f)